In [1]:
from keras.models import *
from keras.layers import *
from keras.applications import *
from keras.preprocessing.image import *

Using TensorFlow backend.


In [2]:
import os

In [3]:
import h5py
import numpy as np
from sklearn.utils import shuffle
np.random.seed(42)

In [4]:
from sklearn.model_selection import KFold

In [250]:
df = pd.read_csv('../driver_imgs_list.csv')
subjects = np.unique(df['subject'])
classnames = np.unique(df['classname'])

In [252]:
kf = KFold(n_splits=5, random_state=42, shuffle=True)
subjects_t = []
subjects_v = []
for sub_t, sub_v in kf.split(subjects):
    subjects_t.append(list(subjects[sub_t]))
    subjects_v.append(list(subjects[sub_v]))

In [259]:
def split_list(subjects_t, subjects_v):
    df_t = df.loc[df.subject.isin(subjects_t)]
    df_v = df.loc[df.subject.isin(subjects_v)]
    return df_t, df_v

In [260]:
df_t1, df_v1 = split_list(subjects_t[0], subjects_v[0])

In [247]:
dataset_dir = '../dataset_original'

dataset_train = dataset_dir + '/train'
dataset_test = dataset_dir + '/test'

In [254]:
for split_name in ['train', 'validation']:
    for cls in classnames:
        os.makedirs(split_name + '/' + cls)

In [263]:
def t_v_split_link(df_t, df_v):
    for i in range(len(df_t)):
        cls = df_t.iloc[i]['classname']
        img = df_t.iloc[i]['img']
        os.symlink(dataset_train+'/'+cls+'/'+img, 'train/'+cls+'/'+img)
    for i in range(len(df_v)):
        cls = df_t.iloc[i]['classname']
        img = df_t.iloc[i]['img']
        os.symlink(dataset_train+'/'+cls+'/'+img, 'validation/'+cls+'/'+img)

In [ ]:
dataset_train = 'train'
dataset_val = 'validation'

In [281]:
def preprocess_fn(x):
    # RGB >> BGR
    x = img_to_array(x)
    x = x[:, :, ::-1]
    # Zero-center by mean pixel
    x[:, :, 0] -= 103.939
    x[:, :, 1] -= 116.779
    x[:, :, 2] -= 123.68
    return x

In [ ]:
def write_tfv(MODEL, image_size):
    width = image_size[0]
    height = image_size[1]
    input_tensor = Input((height, width, 3))
    x = input_tensor
    
    base_model = MODEL(input_tensor=x, weights='imagenet', include_top=True)
    if MODEL = 'ResNet50':
        model = Model(base_model.input, base_model.get_layer('flatten_3').output)
    else:
        model = Model(base_model.input, base_model.get_layer('flatten').output)
    
    print('\n')
    batch_size = 32
    gen = ImageDataGenerator(preprocessing_function=preprocess_fn)
    train_generator = gen.flow_from_directory(train_dir, image_size, shuffle=False, 
                                              batch_size=batch_size)
    val_generator = gen.flow_from_directory(train_dir, image_size, shuffle=False, 
                                              batch_size=batch_size)
    test_generator = gen.flow_from_directory(test_dir, image_size, shuffle=False, 
                                             batch_size=batch_size, class_mode=None)
    
    print('generating training transfer-value')
    train = model.predict_generator(train_generator, train_generator.samples//batch_size+1)
    print('training set shape: {}'.format(train.shape))
    
    print('generating validation transfer-value')
    val = model.predict_generator(val_generator, val_generator.samples//batch_size+1)
    print('validation set shape: {}'.format(val.shape))
    
    print('generating test transfer-value')
    test = model.predict_generator(test_generator, test_generator.samples//batch_size+1)
    print('test set shape: {}'.format(test.shape))
    
    filenames_train = train_generator.filenames
    for i, fname in enumerate(train_generator.filenames):
        filenames_train[i] = os.path.basename(fname).encode()
    filenames_val = val_generator.filenames
    for i, fname in enumerate(val_generator.filenames):
        filenames_val[i] = os.path.basename(fname).encode()
    filenames_test = test_generator.filenames
    for i, fname in enumerate(test_generator.filenames):
        filenames_test[i] = os.path.basename(fname).encode()
        
    print('start writing h5 file.')
    with h5py.File('tfv_{}.h5'.format(MODEL.__name__)) as h:
        h.create_dataset('train', data=train)
        h.creat_dataset('validation', data=val)
        h.create_dataset('test', data=test)
        h.create_dataset('label_t', data=train_generator.classes)
        h.create_dataset('label_v', data=val_generator.classes)
        h.create_dataset('filename_train', data=filenames_train)
        h.create_dataset('filename_val', data=filenames_val)
        h.create_dataset('filename_test', data=filenames_test)

In [ ]:
filename = 

In [ ]:
with h5py.File(filename, 'r') as h:
    X_train = np.array(h['train'])
    X_val = np.array(h['validation'])
    X_test = np.array(h['test'])
    y_train = np.array(h['label_t'])
    y_val = np.array(h['label_v'])
    test_fname = np.array(h['filename_test'])

In [ ]:
input_tensor = Input(X_train.shape[1:])
x = Dropout(0.5)(input_tensor)

x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)

x = Dense(10, activation='softmax')(x)
model = Model(input_tensor, x)
model.compile(optimizer='adadelta',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [6]:
dataset_dir = '../transfer_value_keras/'

In [16]:
data_list = os.listdir(dataset_dir)
for i, file in enumerate(data_list):
    data_list[i] = dataset_dir + file

In [21]:
data_list

['../transfer_value_keras/tfv_InceptionV3.h5',
 '../transfer_value_keras/tfv_ResNet50.h5',
 '../transfer_value_keras/tfv_VGG16.h5',
 '../transfer_value_keras/tfv_VGG19.h5',
 '../transfer_value_keras/tfv_Xception.h5']

In [212]:
X_train = []
X_test = []
for file in data_list:
    with h5py.File(file, 'r') as h:
        X_train.append(np.array(h['train']))
        X_test.append(np.array(h['test']))
        y_train = np.array(h['label'])
        X_fname = np.array(h['train_filename'])
        
X_train = np.concatenate(X_train, axis=1)
X_test = np.concatenate(X_test, axis=1)   

In [215]:
X_train.shape

(22424, 7168)

In [214]:
X_test.shape

(79726, 7168)

In [216]:
def train_val_split(X_train, X_test, y_train, X_fname, val_size=0.2, random_state=42):
    fnames = []
    for i in range(len(X_fname)):
        fnames.append(X_fname[i].decode())
        
    df1 = pd.DataFrame(fnames, columns=['img'])
    df2 = pd.DataFrame(y_train, columns=['label'])
    df3 = pd.DataFrame(X_train)
    df_raw = pd.concat([df1, df2, df3], axis=1)
    
    df = pd.read_csv('../driver_imgs_list.csv')
    subjs = np.unique(df['subject'])
    sub_train, sub_val = train_test_split(subjs, test_size=val_size, random_state=random_state)
    df_t = df.loc[df.subject.isin(sub_train)]
    df_v = df.loc[df.subject.isin(sub_val)]
    img_t = df_t['img']
    img_v = df_v['img']
    
    df_train = df_raw.loc[df_raw.img.isin(img_t)]
    df_val = df_raw.loc[df_raw.img.isin(img_v)]
    
    X_train, y_train, X_val, y_val = np.array(df_train.iloc[:, 2:]), np.array(df_train['label']), np.array(df_val.iloc[:, 2:]), np.array(df_val['label'])
    
    X_train, y_train = shuffle(X_train, y_train)
    X_val, y_val = shuffle(X_val, y_val)
    
    y_train = to_categorical(y_train, num_classes=10)
    y_val = to_categorical(y_val, num_classes=10)
    
    return X_train, y_train, X_val, y_val

In [184]:
fnames = []
for i in range(len(X_fname)):
    fnames.append(X_fname[i].decode())

In [186]:
df = pd.read_csv('../driver_imgs_list.csv')

In [187]:
subjs = np.unique(df['subject'])
subjs

array(['p002', 'p012', 'p014', 'p015', 'p016', 'p021', 'p022', 'p024',
       'p026', 'p035', 'p039', 'p041', 'p042', 'p045', 'p047', 'p049',
       'p050', 'p051', 'p052', 'p056', 'p061', 'p064', 'p066', 'p072',
       'p075', 'p081'], dtype=object)

In [188]:
from sklearn.model_selection import train_test_split
sub_train, sub_val = train_test_split(subjs, test_size=0.2, random_state=42)

In [189]:
df_t = df.loc[df.subject.isin(sub_train)]
df_v = df.loc[df.subject.isin(sub_val)]
img_t = df_t['img']
img_v = df_v['img']

In [190]:
df_train = df_raw.loc[df_raw.img.isin(img_t)]
df_val = df_raw.loc[df_raw.img.isin(img_v)]

In [193]:
X_train, y_train, X_val, y_val = np.array(df_train.iloc[:, 2:]), np.array(df_train['label']), np.array(df_val.iloc[:, 2:]), np.array(df_val['label'])

In [194]:
X_train, y_train = shuffle(X_train, y_train)
X_val, y_val = shuffle(X_val, y_val)

In [195]:
from keras.utils.np_utils import to_categorical

y_train = to_categorical(y_train, num_classes=10)
y_val = to_categorical(y_val, num_classes=10)

In [217]:
X_train, y_train, X_val, y_val = train_val_split(X_train, X_test, y_train, X_fname)

In [221]:
from keras.models import *
from keras.layers import *

input_tensor = Input(X_train.shape[1:])
x = Dropout(0.5)(input_tensor)

x = Dense(128, activation='elu')(x)
x = Dropout(0.5)(x)

x = Dense(10, activation='softmax')(x)
model = Model(input_tensor, x)
model.compile(optimizer='adadelta',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [222]:
model.fit(X_train, y_train, batch_size=32, epochs=50, validation_data=(X_val, y_val), verbose=2)

Train on 17446 samples, validate on 4978 samples
Epoch 1/50
14s - loss: 2.7851 - acc: 0.1714 - val_loss: 2.0325 - val_acc: 0.2485
Epoch 2/50
13s - loss: 1.8047 - acc: 0.4007 - val_loss: 1.7196 - val_acc: 0.3694
Epoch 3/50
13s - loss: 1.2479 - acc: 0.5692 - val_loss: 1.9724 - val_acc: 0.3919
Epoch 4/50
13s - loss: 0.9340 - acc: 0.6773 - val_loss: 1.7358 - val_acc: 0.4454
Epoch 5/50
13s - loss: 0.7672 - acc: 0.7380 - val_loss: 1.9581 - val_acc: 0.4269
Epoch 6/50
13s - loss: 0.6512 - acc: 0.7755 - val_loss: 1.8359 - val_acc: 0.4590
Epoch 7/50
13s - loss: 0.5691 - acc: 0.8051 - val_loss: 1.8137 - val_acc: 0.4652
Epoch 8/50
15s - loss: 0.5101 - acc: 0.8244 - val_loss: 1.9300 - val_acc: 0.4755
Epoch 9/50
16s - loss: 0.4822 - acc: 0.8372 - val_loss: 2.1759 - val_acc: 0.4446
Epoch 10/50
14s - loss: 0.4407 - acc: 0.8502 - val_loss: 1.9453 - val_acc: 0.4588
Epoch 11/50
13s - loss: 0.4050 - acc: 0.8632 - val_loss: 1.8375 - val_acc: 0.4753
Epoch 12/50
13s - loss: 0.3777 - acc: 0.8727 - val_loss: 2

KeyboardInterrupt: 